In [1]:
import gurobipy as gp
from gurobipy import GRB

## Parameters

In [2]:
arcs, capacity = gp.multidict({
    (1, 2): 5,
    (1, 3): 6,
    (1, 4): 5,
    #
    (2, 3): 2,
    (2, 5): 3,
    #
    (3, 2): 2,
    (3, 4): 3,
    (3, 5): 3,
    (3, 6): 7,
    #
    (4, 6): 5,
    #
    (5, 6): 1,
    (5, 7): 8,
    #
    (6, 5): 1,
    (6, 7): 7,
    #
    (7, 1): GRB.INFINITY,
})

In [3]:
nodes = range(1, 8)

# Model
## Decision Variables

In [4]:
m = gp.Model('max_flow')
path = m.addVars(arcs, ub=capacity, vtype=GRB.CONTINUOUS, name='path')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-19


## Objective Function

In [5]:
Z = path[7,1]
m.ModelSense = GRB.MAXIMIZE
m.setObjective(Z)

## Constraints

In [6]:
transshipment = m.addConstrs(
    (path.sum('*', t) - path.sum(t, '*') == 0 for t in nodes),
    name='transshipment'
)

In [7]:
m.update()
print(f'Number of variables: {m.NumVars}')
print(f'Number of constraints: {m.NumConstrs}')
print(f'Constraints are \n {m.getConstrs()}')

Number of variables: 15
Number of constraints: 7
Constraints are 
 [<gurobi.Constr transshipment[1]>, <gurobi.Constr transshipment[2]>, <gurobi.Constr transshipment[3]>, <gurobi.Constr transshipment[4]>, <gurobi.Constr transshipment[5]>, <gurobi.Constr transshipment[6]>, <gurobi.Constr transshipment[7]>]


## Optimization process

In [8]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x282bdc36
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 6 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4008000e+01   2.001600e+01   0.000000e+00      0s
       5    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.400000000e+01


## Results

In [9]:
print(f'Max Flow: {m.objVal * 1000:,.0f} vehicles')

Max Flow: 14,000 vehicles


In [10]:
for v in m.getVars():
    if abs(v.x) > 1e-5:
        print(f'{v.VarName}: {v.x * 1000:,.0f}')

path[1,2]: 5,000
path[1,3]: 4,000
path[1,4]: 5,000
path[2,3]: 2,000
path[2,5]: 3,000
path[3,5]: 3,000
path[3,6]: 3,000
path[4,6]: 5,000
path[5,7]: 7,000
path[6,5]: 1,000
path[6,7]: 7,000
path[7,1]: 14,000
